In [25]:
"""
Created on: Mon. 29 Aug. 2022
Author: Mélina Verger
"""

# For data manipulation
import pandas as pd

# To handle ZIP files
import zipfile


In [26]:
zf = zipfile.ZipFile("./data/data.zip") 

In [29]:
studentInfo = pd.read_csv(zf.open("studentInfo.csv"))
studentAssessment = pd.read_csv(zf.open("studentAssessment.csv"))
studentVle = pd.read_csv(zf.open("studentVle.csv"))